# Install packages and Various Utilities

## Check how much storage left

In [1]:
import psutil
import sys
print(sys.version)

def get_disk_usage():

    # Get disk usage
    disk = psutil.disk_usage('/')
    print("Total Disk Space:", disk.total / (1024 * 1024 * 1024), "GB")
    print("Used Disk Space:", disk.used / (1024 * 1024 * 1024), "GB")
    print("Free Disk Space:", disk.free / (1024 * 1024 * 1024), "GB")
    print("Disk Free Usage Percentage:", 100 - disk.percent, "%")


# before installation
get_disk_usage()

3.10.12 (main, Jan 17 2025, 14:35:34) [GCC 11.4.0]
Total Disk Space: 348.71622467041016 GB
Used Disk Space: 135.84511184692383 GB
Free Disk Space: 212.85548782348633 GB
Disk Free Usage Percentage: 61.0 %


## Check whether GPU is backened up

In [2]:
import torch

# Get the PyTorch version
torch_version = torch.__version__

# Print the version
print(f"PyTorch version: {torch_version}")

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA is available: {cuda_available}")

# If CUDA is available, get more details
if cuda_available:
    # Number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of the current CUDA device
    current_device = torch.cuda.get_device_name(0)
    print(f"Current CUDA device: {current_device}")

    # Check if PyTorch is compiled with CUDA support
    cuda_version = torch.version.cuda
    print(f"CUDA version: {cuda_version}")

    # Check if PyTorch is running on GPU
    tensor = torch.tensor([1.0])
    is_on_gpu = tensor.is_cuda
    print(f"Tensor on GPU: {is_on_gpu}")  # This will be False since we haven't moved the tensor to GPU yet

    # Move tensor to GPU to check if it works
    tensor = tensor.to('cuda')
    is_on_gpu = tensor.is_cuda
    print(f"Tensor moved to GPU: {is_on_gpu}")

else:
    print("No CUDA-capable device found. Running on CPU.")

PyTorch version: 2.5.1+cu124
CUDA is available: True
Number of GPUs available: 4
Current CUDA device: NVIDIA A10G
CUDA version: 12.4
Tensor on GPU: False
Tensor moved to GPU: True


## Download pretrained SAM models

In [7]:
import requests
import os

def download_file(url, destination_folder, filename=None):
    """
    Download a file from the given URL and save it to the specified destination folder.

    :param url: URL of the file to download.
    :param destination_folder: Path to the folder where the file should be saved.
    :param filename: Optional filename to use when saving the file. If None, the filename from the URL will be used.
    :return: The full path of the saved file, or None if there was an error.
    """
    
    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)
    
    # If no filename is provided, extract it from the URL
    if filename is None:
        filename = url.split('/')[-1]
    
    # Full path where the file will be saved
    save_path = os.path.join(destination_folder, filename)
    
    try:
        # Download the file
        with requests.get(url, stream=True) as r:
            r.raise_for_status()  # Raise an HTTPError for bad responses
            with open(save_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192): 
                    f.write(chunk)
        print(f"File downloaded successfully to {save_path}")
        return save_path
    except requests.RequestException as e:
        print(f"Failed to download file: {e}")
        return None

# Example usage
# Replace with your URL
# url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth"  
# url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"
url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"
destination_folder = "/home/ubuntu/SAM1/SAM_Adapter/pretrained"  # Replace with your destination folder
download_file(url, destination_folder)

File downloaded successfully to /home/ubuntu/SAM1/SAM_Adapter/pretrained/sam_vit_b_01ec64.pth


'/home/ubuntu/SAM1/SAM_Adapter/pretrained/sam_vit_b_01ec64.pth'

In [3]:
!wget -O sam2_hiera_tiny.pt "https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt"
!wget -O sam2_hiera_small.pt "https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt"
!wget -O sam2_hiera_base_plus.pt "https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt"
!wget -O sam2_hiera_large.pt "https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt"

--2025-01-22 15:27:56--  https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.222.236.117, 52.222.236.74, 52.222.236.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.222.236.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155906050 (149M) [application/vnd.snesdev-page-table]
Saving to: ‘sam2_hiera_tiny.pt’

sam2_hiera_tiny.pt  100%[===================>] 148.68M   260MB/s    in 0.6s    

2025-01-22 15:27:57 (260 MB/s) - ‘sam2_hiera_tiny.pt’ saved [155906050/155906050]

--2025-01-22 15:27:57--  https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.222.236.74, 52.222.236.117, 52.222.236.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.222.236.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184309650 (176M) [app

## Upzip a file to a destination

In [8]:
import zipfile
import os

def unzip_file(zip_path, extract_to):
    """
    Unzip a file to a specified destination.

    :param zip_path: Path to the zip file.
    :param extract_to: Directory where to extract the contents.
    """
    # Ensure the directory exists
    os.makedirs(extract_to, exist_ok=True)
    
    # Open the zip file and extract its contents
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    print(f"{zip_path} was upzipped.")

# Example usage
zip_file_path = '/home/ubuntu/SAM1/DataExample4_Processed.zip'
destination_path = '/home/ubuntu/SAM1/Data'
unzip_file(zip_file_path, destination_path)

## Zip a file to a destination

In [ ]:
import zipfile
import os

def zip_directory(directory_to_zip, zip_destination):
    """
    Zip all files within a directory to a specified destination.

    :param directory_to_zip: Path to the directory whose contents are to be zipped.
    :param zip_destination: Path where to save the zip file.
    """
    # Ensure the directory for the zip file exists
    os.makedirs(os.path.dirname(zip_destination), exist_ok=True)
    
    with zipfile.ZipFile(zip_destination, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, dirs, files in os.walk(directory_to_zip):
            for file in files:
                # Construct full file path
                file_path = os.path.join(root, file)
                # Write the file to the zip archive, maintaining relative path
                zip_file.write(file_path, os.path.relpath(file_path, directory_to_zip))

# Example usage
directory_to_zip = 'path/to/your/directory'
zip_destination = 'path/to/your/destination/yourdirectory.zip'
zip_directory(directory_to_zip, zip_destination)

## Install packages from the requirements.txt

In [11]:
!pip install -r requirements_aws.txt

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.6/593.6 kB 46.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... one
  Preparing metadata (setup.py) ... one
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 15.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 126.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!git clone https://github.com/facebookresearch/sam2.git

Cloning into 'sam2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 134.70 MiB | 43.43 MiB/s, done.
Resolving deltas: 100% (375/375), done.
